# Learning Goals
In Assignment 2, we learnt how to construct networks of spiking neurons and propagate information through a network of fixed weights. In this assignment, you will learn how to train network weights for a given task using brain-inspired learning rules.

Let's import all the libraries required for this assignment. 

In [6]:
import math
import numpy as np
import matplotlib.pyplot as plt
import copy

# Question 1: Training a Network

## 1a. 
What is the purpose of a learning algorithm? In other words, what does a learning algorithm dictate, and what is the objective of it?

## Answer 1a. 
The learning algorithms helps us train a machine learning model and make predictions based on that learning. It also dictates how a model updates its parameters or weights based on the input data to reduce the difference between the models outputs and targeted outputs or actual outputs. The target of a learning algorithm is to reduce the prediction error on new or unseen data. In other words, the purpose of the learning algorithm is to make the model learn on the input data and improve its performance without being explicitly programmed to do so. A learning algorithm specifies how and under what conditions a learning rule or a combination of learning rules should be applied to adjust the network parameters.

## 1b. 
Categorize and explain the various learning algorithms w.r.t. biological plausibility. Can you explain the tradeoffs involved with the different learning rules? *Hint: Think computational advantages and disadvantages of biological plausibility.*

## Answer 1b. 
The learning algorithms can be divided into two categories one with rules that are biologically plausible and others that are not-biologically plausible:

BIOLOGICAL PLAUSIBILITY: In Brain Inspired Computing biological plausibility means that the computational model or algorithm must be based on principles that are consistent with what we know as the functioning of biological neurons andetworks in the brain.

a) Biological Plausible: 
- The weight change between a presynaptic and a postsynaptic neuron should only depend upon pre and post synaptic neuron joint activity, and also satisfy the constraint of locality.
- Hebbian Learning algorithm and Spike Time Dependent Plasticity (STDP) are the algorithms which are biologically plausible.
- Hebb's rule, Oja's rule, BCM rule and Covariance rules are biologically plausible rules. 
- Hebb's rule considers correlation between the pre and post synaptic firing rates. And with each epoch or iteration the weights keep growing. In a system where synapses can only be strengthened all efficacies will eventually saturate at their upper maximum value. And hebbain cannot learn on negative weights.
- Oja’s rule converges asymptotically to synaptic weights that are normalized to ∑<sub>𝑗</sub>W<sup>2</sup><sub>𝑖𝑗</sub> = 1 while keeping the essential Hebbian properties of the standard rule. We note that normalization of ∑<sub>𝑗</sub>W<sup>2</sup><sub>𝑖𝑗</sub> implies competition between the synapses that make connections to the same postsynaptic neuron, i.e., if some weights grow, others must decrease.
- In Bienenstock-Cooper-Munro (BCM) rule the synaptic plasticity is characterized by two threshold for the postsynaptic activity, v<sub>0</sub> and v<sub>$\theta$</sub>. If presynaptic activity is combined with moderate levels of postsynaptic excitation, the efficacy of synapses activated by presynaptic input is decreased . Weights are increased only if the level of postsynaptic activity exceeds a threshold, v<sub>$\theta$</sub>. The change of weights is restricted to those synapses which are activated by presynaptic inputBelow v<sub>0</sub> no synaptic modification occurs, between v<sub>0</sub> and v<sub>$\theta$</sub> synapses are depressed, and for postsynaptic firing rates beyond v<sub>$\theta$</sub> synaptic potentiation can be observed. The BCM rule leads to input selectivity.
- The covariance rule calculates the covariance between the presynaptic and postsynaptic activities and uses this value to update the weight of the synapse. Specifically, the change in the weight of the synapse is proportional to the covariance between the presynaptic and postsynaptic activities. It is based on the idea that rates v<sub>i</sub> and v<sub>j</sub> fluctuate around mean values <v<sub>i</sub>>,<v<sub>j</sub>> that are taken as running averages over the recent firing history.
- STDP's main idea is that it is the timing of spikes rather than the specific rate of spikes that carries neural information. This observation is indeed in agreement with Hebb’s postulate because presynaptic neurons that are active slightly before the postsynaptic neuron are those which take part in firing it whereas those that fire later did not contribute to the postsynaptic action potential.

b) Non-Biologically palusible:
- Non-Local connections can influence the wight change.
- Backpropagation using Gardient Descent is Non-biologically plausible algorithm.
- The backpropagation algorithm works by iteratively adjusting the weights of the connections between neurons in the network based on the error between the predicted output and the actual output. This process involves two main steps: forward propagation and backpropagation.

In the forward propagation step, the input data is passed through the network and the output is calculated using the current weights of the connections between the neurons. This output is compared to the actual output, and the error between the two is calculated.

In the backpropagation step, the error is propagated backwards through the network from the output layer to the input layer. The weights of the connections between the neurons are then adjusted based on the calculated error using gradient descent.




# Question 2: Hebbian Learning

## 2a.

In this exercise, you will implement the hebbian learning rule to solve AND Gate. First, we need to create a helper function to generate the training data. The function should return lists of tuples where each tuple comprises of numpy arrays of rate-coded inputs and the corresponding rate-coded output. 

Below is the function to generate the training data. Fill the components to return the training data. 

In [2]:
def genANDTrainData(snn_timestep):
    """ 
    Function to generate the training data for AND 
        Args:
            snn_timestep (int): timesteps for SNN simulation
        Return:
            train_data (list): list of tuples where each tuple comprises of numpy arrays of rate-coded inputs and output
        
        Write the expressions for encoding 0 and 1. Then append all 4 cases of AND gate to the list train_data
    """
    
    #Initialize an empty list for train data
    train_data = []
    
    #encode 0. Numpy random choice function might be useful here.
    encode_0 = np.random.choice([0,1],snn_timestep, p=[0.7,0.3]) 

    
    #encode 1. Numpy random choice function might be useful here.
    encode_1 = np.random.choice([0,1],snn_timestep, p=[0.3,0.7])  

    y = np.logical_and(encode_0, encode_1)
    #Append all 4 cases of AND gate to train_data. Numpy stack operation might be useful here. 
    # e.g., [0,0], [1,0], [0,1], [1,1]
    and_00 = [np.array([encode_0, encode_0]), encode_0]
    and_01 = [np.array([encode_0, encode_1]), encode_0]
    and_10 = [np.array([encode_1, encode_0]), encode_0]
    and_11 = [np.array([encode_1, encode_1]), encode_1]
    train_data = np.stack((and_00, and_01, and_10, and_11))


    return train_data

## 2b. 
We will use the implementation of the network from assignment 2 to create an SNN comprising of one input layer and one output layer. Can you explain algorithmically, how you can use this simple architecture to learn AND gate. Your algorithm should comprise of encoding, forward propagation, network training, and decoding steps. 

## Answer 2b. 
The algorithm consists of multiple steps like encoding, training, forward propagation and decoding:<br>

1. Encoding:
We encode 0 and 1 using rate-based encoding<br>Generate train dataset using the encoded values of 0 and 1 for all cases of AND gate:
    - Case 1: 0 & 0 = 0
    - Case 2: 0 & 1 = 0
    - Case 3: 1 & 0 = 0
    - Case 4: 1 & 1 = 1
2. Training:
Initialize the network with random weights
    - Iterate over all examples in the training dataset: [[X<sub>1</sub>,X<sub>2</sub>],Y]
        - Calculate Input Firing Rate for X<sub>1</sub>: r<sub>1</sub>
        - Calculate Input Firing Rate for X<sub>2</sub>: r<sub>2</sub>
        - Calculate Output Firing Rate for Y: r<sub>o</sub>
        - Calculate correlation: corr<sub>X<sub>1</sub>,Y</sub> = r<sub>1</sub>*r<sub>o</sub>; &nbsp; &nbsp; corr<sub>X<sub>2</sub>,Y</sub> = r<sub>2</sub>*r<sub>o</sub>
        - Update weight: w<sub>11</sub> = w<sub>11</sub> + $\eta $ * corr<sub>X<sub>1</sub>,Y</sub>; &nbsp; &nbsp; w<sub>22</sub> = w<sub>22</sub> + $\eta $ * corr<sub>X<sub>2</sub>,Y</sub>
    - Repeat for specified number of epochs
3. Forward Propagation:

- Iterate over train dataset to propagate spikes through network using trained weights for the network<br>
- For each training instance repeat for specified number of timesteps and accumulate output spikes, calculate output firing rate at the end of timestep
4. Decoding:Decode the rate into classes [0, 1] using thresholding

The SNN has already been implemented for you. You do not need to do anything here. Just understand the implementation so that you can use it in the later parts. 

In [3]:
class LIFNeurons:
    """ 
        Define Leaky Integrate-and-Fire Neuron Layer 
        This class is complete. You do not need to do anything here.
    """

    def __init__(self, dimension, vdecay, vth):
        """
        Args:
            dimension (int): Number of LIF neurons in the layer
            vdecay (float): voltage decay of LIF neurons
            vth (float): voltage threshold of LIF neurons
        
        """
        self.dimension = dimension
        self.vdecay = vdecay
        self.vth = vth

        # Initialize LIF neuron states
        self.volt = np.zeros(self.dimension)
        self.spike = np.zeros(self.dimension)
    
    def __call__(self, psp_input):
        """
        Args:
            psp_input (ndarray): synaptic inputs 
        Return:
            self.spike: output spikes from the layer
                """
        self.volt = self.vdecay * self.volt * (1. - self.spike) + psp_input
        self.spike = (self.volt > self.vth).astype(float)
        return self.spike

class Connections:
    """ Define connections between spiking neuron layers """

    def __init__(self, weights, pre_dimension, post_dimension):
        """
        Args:
            weights (ndarray): connection weights
            pre_dimension (int): dimension for pre-synaptic neurons
            post_dimension (int): dimension for post-synaptic neurons
        """
        self.weights = weights
        self.pre_dimension = pre_dimension
        self.post_dimension = post_dimension
    
    def __call__(self, spike_input):
        """
        Args:
            spike_input (ndarray): spikes generated by the pre-synaptic neurons
        Return:
            psp: postsynaptic layer activations
        """
        psp = np.matmul(self.weights, spike_input)
        return psp
    
    
class SNN:
    """ Define a Spiking Neural Network with No Hidden Layer """

    def __init__(self, input_2_output_weight, 
                 input_dimension=2, output_dimension=2,
                 vdecay=0.5, vth=0.5, snn_timestep=20):
        """
        Args:
            input_2_hidden_weight (ndarray): weights for connection between input and hidden layer
            hidden_2_output_weight (ndarray): weights for connection between hidden and output layer
            input_dimension (int): input dimension
            hidden_dimension (int): hidden_dimension
            output_dimension (int): output_dimension
            vdecay (float): voltage decay of LIF neuron
            vth (float): voltage threshold of LIF neuron
            snn_timestep (int): number of timesteps for inference
        """
        self.snn_timestep = snn_timestep
        self.output_layer = LIFNeurons(output_dimension, vdecay, vth)
        self.input_2_output_connection = Connections(input_2_output_weight, input_dimension, output_dimension)
    
    def __call__(self, spike_encoding):
        """
        Args:
            spike_encoding (ndarray): spike encoding of input
        Return:
            spike outputs of the network
        """
        spike_output = np.zeros(self.output_layer.dimension)
        for tt in range(self.snn_timestep):
            input_2_output_psp = self.input_2_output_connection(spike_encoding[:, tt])
            output_spikes = self.output_layer(input_2_output_psp)
            spike_output += output_spikes
        return spike_output/self.snn_timestep      

## 2c. 
Next, you need to write a function for network training using hebbian learning rule. The function is defined below. You need to fill in the components so that the network weights are updated in the right manner. 

In [12]:
def hebbian(network, train_data, lr=1e-3, epochs=10):
    """ 
    Function to train a network using Hebbian learning rule
        Args:
            network (SNN): SNN network object
            train_data (list): training data 
            lr (float): learning rate
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples. 
        
        Write the operations required to compute the weight increment according to the hebbian learning rule. Then increment the network weights. 
    """
    
    #iterate over the epochs
    for ee in range(epochs):
        #iterate over all samples in train_data
        for data in train_data:
            #compute the firing rate for the input
            v_i = np.sum(data[0])/len(data[1])
            #compute the firing rate for the output
            v_j = np.sum(data[1])/len(data[1])
            #compute the correlation using the firing rates calculated above
            correlation = v_i * v_j
            #compute the weight increment
            delta_w = lr * correlation
            #increment the weight
            network.input_2_output_connection.weights += delta_w

## 2d. 
In this exercise, you will use your implementations above to train an SNN to learn AND gate. 

In [64]:
#Define a variable for input dimension
input_dim = 2

#Define a variable for output dimension
out_dim = 1

#Define a variable for voltage decay
vdecay=0.3

#Define a variable for voltage threshold
vth=0.2

#Define a variable for snn timesteps
snn_timestep=10

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here. 
_weight = np.random.rand(out_dim, input_dim)
input_2_output_weight = copy.copy(_weight)
#print the initial weights
print("Initial weights:",input_2_output_weight)

#Initialize an snn using the arguments defined above
snn = SNN(input_2_output_weight, input_dim, out_dim, vdecay, vth, snn_timestep)
#Get the training data for AND gate using the function defined in 2a. 
train_data = genANDTrainData(snn_timestep)
#Train the network using the function defined in 2c. with the appropriate arguments
hebbian(snn, train_data)

pred=[]
#Test the trained network and print the network output for all 4 cases. 
for x in range(len(train_data)):
    print("Interation "+str(x)+":")
    print(train_data[x][0])
    out = snn(train_data[x][0])
    if out[0] >= 0.6:
        pred.append(1)
    else:
        pred.append(0)
    print("Output Spiking Rate:",str(out))
print("\nPrediction: ", str(pred))
    
#Print Final Network Weights
print("Final weights = ", str(snn.input_2_output_connection.weights))

Initial weights: [[0.06108384 0.10768687]]
Interation 0:
[[1 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 1 0 0 0]]
Output Spiking Rate: [0.2]
Interation 1:
[[1 0 0 0 0 0 1 0 0 0]
 [1 1 1 1 1 1 0 1 0 1]]
Output Spiking Rate: [0.1]
Interation 2:
[[1 1 1 1 1 1 0 1 0 1]
 [1 0 0 0 0 0 1 0 0 0]]
Output Spiking Rate: [0.1]
Interation 3:
[[1 1 1 1 1 1 0 1 0 1]
 [1 1 1 1 1 1 0 1 0 1]]
Output Spiking Rate: [0.8]

Prediction:  [0, 0, 0, 1]
Final weights =  [[0.07868384 0.12528687]]


# Question 3: Limitations of Hebbian Learning rule

## 3a. 
Can you learn the AND gate using 2 neurons in the output layer instead of one? If yes, describe what changes you might need to make to your algorithm in 2b. If not, explain why not, and what consequences it might entail for the use of hebbian learning for complex real-world tasks. 

## Answer 3a. 
When the output layer has two neurons, it can represent the absence or presence of each potential output (either 0 or 1) for the AND gate. However, this interpretation doesn't match well with the fundamental Hebbian learning rule, which doesn't optimize for probabilities but instead models the correlation between input and output activity.

The Hebbian learning rule has limitations because it can't learn negative weights, resulting in weight saturation and reduced selectivity. In the case of a two-neuron output layer, this restriction implies that the input-output neuron connections will grow and eventually saturate, making it difficult for the network to learn the AND gate correctly.

If we want to use two output neurons for the AND gate, we must adjust the weights and connections accordingly. Specifically, we would need four weights connecting each input neuron to each output neuron, and we would need to establish connections between the existing neurons and the additional neuron in the output layer.

## 3b. 
Train the network using hebbian learning for AND gate with the same arguments as defined in 2d. but now multiply the number of epochs by 20. Can your network still learn AND gate correctly? Inspect the initial and final network weights, and compare them against the network weights in 2d. Based on this, explain your observations for the network behavior. 

In [134]:
#Implementation for 3b. (same as 2d. but with change of one argument)

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here.
in_2_out_weight = np.random.rand(out_dim, input_dim)

#print the initial weights
print("Initial weights = ", str(in_2_out_weight))

#Initialize an snn using the arguments defined above
snn_n = SNN(in_2_out_weight, input_dim, out_dim, vdecay, vth, snn_timestep)

#Get the training data for AND gate using the function defined in 2a. 
train_data = genANDTrainData(snn_timestep)

#Train the network using the function defined in 2c. with the appropriate arguments
hebbian(snn_n, train_data, lr=1e-3, epochs=10*20)

pred_n = []
#Test the trained network and print the network output for all 4 cases.
for ii in range(len(train_data)):
    print(str(ii)+":") 
    print(train_data[ii][0])
    out = snn_n(train_data[ii][0])
    if out[0] >= 0.6:
        pred_n.append(1)
    else:
        pred_n.append(0)
    print("Output Spiking Rate:", str(out))
print("\nPrediction: ", str(pred_n))

#Print Final Network Weights
print("Final weights = ", str(snn_n.input_2_output_connection.weights))





Initial weights =  [[0.13681335 0.17209685]]
0:
[[0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0]]
Output Spiking Rate: [0.25]
1:
[[0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0]
 [1 0 0 0 1 1 0 0 1 1 1 0 1 0 1 1 1 1 1 1]]
Output Spiking Rate: [0.35]
2:
[[1 0 0 0 1 1 0 0 1 1 1 0 1 0 1 1 1 1 1 1]
 [0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0]]
Output Spiking Rate: [0.4]
3:
[[1 0 0 0 1 1 0 0 1 1 1 0 1 0 1 1 1 1 1 1]
 [1 0 0 0 1 1 0 0 1 1 1 0 1 0 1 1 1 1 1 1]]
Output Spiking Rate: [0.65]

Prediction:  [0, 0, 0, 1]
Final weights =  [[0.42081335 0.45609685]]


## Answer 3b. 
Increasing the number of epochs to 20 times the previous attempt and training the network using the Hebbian learning rule results in an incorrect learning of the AND gate. The weights of the network keep increasing as the epochs increase, causing more spiking activity and incorrect classification of inputs.





## 3c. 
Based on your observations and response in 3b., can you explain another limitation of hebbian learning rule w.r.t. weight growth? Can you also suggest a possible remedy for it?

## Answer 3c. 
The Hebbian learning rule has a limitation in that there is no mechanism for weight decay, and weights can grow uncontrollably. Eventually, the weights will saturate in relation to the membrane properties of LIF, and the network may not be as effective at learning new information. To overcome this, one possible solution is to use Oja's rule, which asymptotically converges to synaptic weights that are normalized to $\sum_{j} w_{ij}^{2}=1$. This normalization ensures that if some weights increase, others must decrease, preventing saturation of weights.





## 3d. 
To resolve the issues with hebbian learning, one possibility is Oja's rule. In this exercise, you will implement and train an SNN using Oja's learning rule. 

References: https://neuronaldynamics.epfl.ch/online/Ch19.S2.html

In [67]:
def oja(network, train_data, lr=1e-5, epochs=10):
    """ 
    Function to train a network using Hebbian learning rule
        Args:
            network (SNN): SNN network object
            train_data (list): training data 
            lr (float): learning rate
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples. 
        
        Write the operations required to compute the weight increment according to the hebbian learning rule. Then increment the network weights. 
    """
    
    #iterate over the epochs
    for ee in range(epochs):
        #iterate over all samples in train_data
        for data in train_data:
            #compute the firing rate for the input
            oja_v_i = np.sum(data[0], axis=1)/len(data[1])
            #compute the firing rate for the output
            oja_v_j = np.sum(data[1])/len(data[1])
            #compute the weight increment
            oja_delta_w = lr * oja_v_j * (oja_v_i - (network.input_2_output_connection.weights * oja_v_j))
            #increment the weight
            network.input_2_output_connection.weights += oja_delta_w


Now, test your implementation below. 

In [69]:
#Define a variable for input dimension
oja_input_dimension = 2
#Define a variable for output dimension
oja_output_dimension = 1
#Define a variable for voltage decay
vdecay = 0.5
#Define a variable for voltage threshold
vth = 0.5
#Define a variable for snn timesteps
snn_timestep = 20
#Initialize randomly the weights from input to output. Numpy random rand function might be useful here. 
oja_input_2_output_weight = np.random.rand(oja_output_dimension, oja_input_dimension)
#print the initial weights
print("Initial weights = ", oja_input_2_output_weight)

#Initialize an snn using the arguments defined above
oja_snn = SNN(oja_input_2_output_weight, oja_input_dimension, oja_output_dimension, vdecay, vth, snn_timestep)
#Get the training data for AND gate using the function defined in 2a. 
oja_train_data = genANDTrainData(snn_timestep)
#Train the network using the function defined in 3d. with the appropriate arguments
oja(oja_snn, oja_train_data)

oja_pred =[]
#Test the trained network and print the network output for all 4 cases. 
for x in range(len(oja_train_data)):
    print(str(x)+":") 
    print(oja_train_data[x][0])
    oja_out = oja_snn(oja_train_data[x][0])
    if oja_out[0] >= 0.6:
        oja_pred.append(1)
    else:
        oja_pred.append(0)
    print("Output Spiking Rate:",str(oja_out))
print("\nPrediction: ", str(oja_pred))

#Print Final Network Weights
print("Final weights = ", str(oja_snn.input_2_output_connection.weights))

Initial weights =  [[0.26998317 0.96383494]]
0:
[[1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]]
Output Spiking Rate: [0.3]
1:
[[1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]
 [1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1]]
Output Spiking Rate: [0.8]
2:
[[1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]]
Output Spiking Rate: [0.35]
3:
[[1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1]
 [1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1]]
Output Spiking Rate: [0.8]

Prediction:  [0, 1, 0, 1]
Final weights =  [[0.2700646  0.96385323]]


# Question 4: Spike-time dependent plasticity (STDP)

Reference: https://neuronaldynamics.epfl.ch/online/Ch19.S5.html

## 4a. 
What is the limitation with hebbian learning that STDP aims to resolve?


## Answer 4a. 
Hebbian learning is based on the principle of synaptic plasticity. Hebb's rule considers correlation between the pre and post synaptic firing rates. However, it's limitation is that it does not take into account the timing of the pre and post dynaptic spikes, which can lead to inefficient learning conditions. Whereas, STDP takes into account the precise timing of the spikes in the pre and post synaptic neurons. A synapse is strengthened between two neurons when th epre-synaptic neuron fires hortly before the post-synaptic neuron, and weakens the synapse when the post-synaptic neuron fires shortly before the pre-synaptic neuron. STDP allows the network to learn more precisely when to activate specific synapses. Another drawback of hebbian learning is that the networks trained on it forget the previously learned information when learning new information. STDP addresses this by allowing the network to selectively strengthen or weaken specific synapses based on their activity, rather than overwriting all of the synapses in the network during each learning cycle.






## 4b. 
Describe the algorithm to train a network using STDP learning rule. You do not need to describe encoding here. Your algorithm should be such that its naturally translatable to a program. 

## Answer 4b. 
- STDP's main idea is that it is the timing of spikes rather than the specific rate of spikes that carries neural information.
> - Each spike leaves a trace.
> - At the moment of a presynaptic spike, a decrease of the weight is induced proportional to the value of the postsynaptic trace.
> - Analogously, at the moment of a postsynaptic spike, potentiation of the weight occurs, which is proportional to the presynaptic trace left by a previous presynaptic spike
<br><br>
>   - Initailise the network with random weights.
>   - If a presynaptic spike takes place right before postsynaptic spike, the presynaptic trace at the time of postsynaptic spike is high. So, weight increase will be high. (corresponds to $\Delta t \ge 0$)
>   - If a presynaptic spike takes place right after postsynaptic spike, the presynaptic trace plays no role in the postsynaptic spike. So, weight decrease will be high (corresponds to $\Delta t < 0$)
>   - Below equation indicates the respective increase and decrease in weights in accordance with spike trace:
<br>$\Delta w = A$<sup>+</sup>$exp(-\Delta t/\tau$<sub>p</sub>) &nbsp;&nbsp;$if \Delta t \ge 0$
<br>$\Delta w = -A$<sup>-</sup>$exp(-\Delta t/\tau$<sub>m</sub>) &nbsp;&nbsp;$if \Delta t < 0$ 
>   - Update the weights in accordance with $\Delta w$:
<br>w<sub>new</sub> = w<sub>old</sub> + $\alpha \Delta w$(w<sub>max</sub> - w<sub>old</sub>), &nbsp;&nbsp; $if \Delta w > 0$
<br>w<sub>new</sub> = w<sub>old</sub> + $\alpha \Delta w$(w<sub>old</sub> - w<sub>min</sub>), &nbsp;&nbsp; $if \Delta w \le 0$
>   - Repeat for specified number of epochs.

## 4c. 
In this exercise, you will implement the STDP learning algorithm to train a network. STDP has many different flavors. For this exercise, we will use the learning rule defined in: https://dl.acm.org/doi/pdf/10.1609/aaai.v33i01.330110021. Pay special attention to Equations 2 and 3. 

Below is the class definition for STDP learning algorithm. Your task is to fill in the components so that the weights are updated in the right manner. 

In [70]:
class STDP():
    """Train a network using STDP learning rule"""
    def __init__(self, network, A_plus, A_minus, tau_plus, tau_minus, lr, snn_timesteps=20, epochs=30, w_min=0, w_max=1):
        """
        Args:
            network (SNN): network which needs to be trained
            A_plus (float): STDP hyperparameter
            A_minus (float): STDP hyperparameter
            tau_plus (float): STDP hyperparameter
            tau_minus (float): STDP hyperparameter
            lr (float): learning rate
            snn_timesteps (int): SNN simulation timesteps
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples.  
            w_min (float): lower bound for the weights
            w_max (float): upper bound for the weights
        """
        self.network = network
        self.A_plus = A_plus
        self.A_minus = A_minus
        self.tau_plus = tau_plus
        self.tau_minus = tau_minus
        self.snn_timesteps = snn_timesteps
        self.lr = lr
        self.time = np.arange(0, self.snn_timesteps, 1)
        self.sliding_window = np.arange(-4, 4, 1) #defines a sliding window for STDP operation. 
        self.epochs = epochs
        self.w_min = w_min
        self.w_max = w_max
    
    def update_weights(self, t, i):
        """
        Function to update the network weights using STDP learning rule
        
        Args:
            t (int): time difference between postsynaptic spike and a presynaptic spike in a sliding window
            i(int): index of the presynaptic neuron
        
        Fill the details of STDP implementation
        """
        #compute delta_w for positive time difference
        if t>0:
            delta_w = self.A_plus * math.exp((-1)*t/self.tau_plus)

        #compute delta_w for negative time difference
        else:
            delta_w = -self.A_minus * math.exp((-1)*t/self.tau_minus)

            
        #update the network weights if weight increment is negative
        if delta_w < 0:
            self.network.input_2_output_connection.weights[:,i] += self.lr * delta_w *(self.network.input_2_output_connection.weights[:,i] - self.w_min)

        #update the network weights if weight increment is positive
        elif delta_w > 0:
            self.network.input_2_output_connection.weights[:,i] += self.lr * delta_w *(self.w_max - self.network.input_2_output_connection.weights[:,i])


            
    def train_step(self, train_data_sample):
        """
        Function to train the network for one training sample using the update function defined above. 
        
        Args:
            train_data_sample (list): a sample from the training data
            
        This function is complete. You do not need to do anything here. 
        """
        input = train_data_sample[0]
        output = train_data_sample[1]
        for t in self.time:
            if output[t] == 1:
                for i in range(2):
                    for t1 in self.sliding_window:
                        if (0<= t + t1 < self.snn_timesteps) and (t1!=0) and (input[i][t+t1] == 1):
                            self.update_weights(t1, i)
    
    def train(self, training_data):
        """
        Function to train the network
        
        Args:
            training_data (list): training data
        
        This function is complete. You do not need to do anything here. 
        """
        for ee in range(self.epochs):
            for train_data_sample in training_data:
                self.train_step(train_data_sample)

Let's test the implementation

In [73]:
#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension =1

#Define a variable for voltage decay
vdecay = 0.5

#Define a variable for voltage threshold
vth = 0.5

#Define a variable for snn timesteps
snn_timestep = 20

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here.
ip_2_op_weight = np.random.rand(output_dimension, input_dimension) 
print("Initial Weights = ", str(ip_2_op_weight))

#Initialize an snn using the arguments defined above
snn_stdp = SNN(ip_2_op_weight, input_dimension, output_dimension, vdecay, vth, snn_timestep)

#Get the training data for AND gate using the function defined in 2a.
hebbian_train_data = genANDTrainData(snn_timestep)

#Create an object of STDP class with appropriate arguments
stdp = STDP(snn_stdp, 0.8, 0.2, 3, 0.8, 1e-5, snn_timestep, epochs=20)

#Train the network using STDP
stdp.train(hebbian_train_data)

stdp_pred = []
#Test the trained network and print the network output for all 4 cases. 
for ii in range(len(hebbian_train_data)):
    print(str(ii)+":") 
    print(hebbian_train_data[ii][0])
    stdp_out = snn_stdp(hebbian_train_data[ii][0])
    if stdp_out[0] >= 0.6:
        stdp_pred.append(1)
    else:
        stdp_pred.append(0)
    print("Output Spiking Rate:",str(stdp_out))
print("\nPrediction: ", str(stdp_pred))

print("Final weights = ", str(snn_stdp.input_2_output_connection.weights))

Initial Weights =  [[0.88287353 0.17866873]]
0:
[[0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1]]
Output Spiking Rate: [0.25]
1:
[[0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1]
 [0 0 1 1 1 1 1 0 1 0 1 0 1 0 0 1 0 1 1 1]]
Output Spiking Rate: [0.25]
2:
[[0 0 1 1 1 1 1 0 1 0 1 0 1 0 0 1 0 1 1 1]
 [0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1]]
Output Spiking Rate: [0.6]
3:
[[0 0 1 1 1 1 1 0 1 0 1 0 1 0 0 1 0 1 1 1]
 [0 0 1 1 1 1 1 0 1 0 1 0 1 0 0 1 0 1 1 1]]
Output Spiking Rate: [0.6]

Prediction:  [0, 0, 1, 1]
Final weights =  [[0.82019287 0.16789911]]


# Question 5: OR Gate
Can you train the network with the same architecture in Q2-4 for learning the OR gate. You will need to create another function called genORTrainData. Then create an SNN and train it using STDP. 

In [75]:
#Write your implementation of genORTrainData here. 

def genORTrainData(snn_timestep):
    """ 
    Function to generate the training data for OR 
        Args:
            snn_timestep (int): timesteps for SNN simulation
        Return:
            train_data (list): list of tuples where each tuple comprises of numpy arrays of rate-coded inputs and output
        
        Write the expressions for encoding 0 and 1. Then append all 4 cases of OR gate to the list train_data
    """
    
    #Initialize an empty list for train data
    train_data = []
    
    #encode 0
    x1 = np.random.choice([0,1], snn_timestep, True, [0.7,0.3])
    
    #encode 1
    x2 = np.random.choice([0,1], snn_timestep, True, [0.3,0.7])
    
    #Append all 4 cases of OR gate to train_data
    cs_00 = [np.array([x1,x1]),x1]
    cs_01 = [np.array([x1,x2]),x2]
    cs_10 = [np.array([x2,x1]),x2]
    cs_11 = [np.array([x2,x2]),x2]
    train_data = np.stack((cs_00, cs_01, cs_10, cs_11))

    return train_data



In [76]:
#Train the network for OR gate here using the implementation from 4c. 

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here.
ip_2_op_weight = np.random.rand(output_dimension, input_dimension) 
print("Initial Weights = ", str(ip_2_op_weight))

#Initialize an snn using the arguments defined above
snn_OR_stdp = SNN(ip_2_op_weight, input_dimension, output_dimension, vdecay, vth, snn_timestep)

#Get the training data for AND gate using the function defined in 2a.
hebbian_OR_data = genORTrainData(snn_timestep)

#Create an object of STDP class with appropriate arguments
OR_stdp = STDP(snn_OR_stdp, 0.8, 0.3, 2, 0.9, 1e-5, snn_timestep, epochs=30)

#Train the network using STDP
OR_stdp.train(hebbian_OR_data)

stdp_ORpred = []
#Test the trained network and print the network output for all 4 cases. 
for ii in range(len(hebbian_OR_data)):
    print(str(ii)+":") 
    print(hebbian_OR_data[ii][0])
    stdp_ORout = snn_OR_stdp(hebbian_OR_data[ii][0])
    if stdp_ORout[0] >= 0.6:
        stdp_ORpred.append(1)
    else:
        stdp_ORpred.append(0)
    print("Output Spiking Rate:",str(stdp_ORout))
print("\nPrediction: ", str(stdp_ORpred))

print("Final weights = ", str(snn_OR_stdp.input_2_output_connection.weights))


Initial Weights =  [[0.30869925 0.83766268]]
0:
[[1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]]
Output Spiking Rate: [0.2]
1:
[[1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
 [0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1]]
Output Spiking Rate: [0.8]
2:
[[0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1]
 [1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]]
Output Spiking Rate: [0.2]
3:
[[0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1]
 [0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1]]
Output Spiking Rate: [0.8]

Prediction:  [0, 1, 0, 1]
Final weights =  [[0.23484473 0.62648191]]
